# Libraries and dataset import

In [1]:
%matplotlib inline
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
warnings.simplefilter("ignore", DeprecationWarning)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pickle

from sklearn.model_selection import GridSearchCV

In [2]:
# Interactive visualisation for evaluation of LDA model 
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

C:\Users\eleouetd01\anaconda\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [3]:
amazon2017_string=pd.read_pickle('../amazon2017_string.pkl')

In [4]:
samsung=amazon2017_string[amazon2017_string.brand == 'Samsung']

# VECTORS :Count vectorizer & Tf-IDF Vectorizer

In [5]:
# Before we can apply LDA or NMF, we need to create vocabulary of all the words in our data, 
# vectorized matrix of the vocabulary.

# COUNTVECTORIZER
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, strip_accents='ascii', stop_words='english')
doc_term_matrix_cv = count_vectorizer.fit_transform(samsung.text_processed) # vocabulary encoded into vectors with countvectorizer

# IF-IDF VECTORIZER
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, strip_accents='ascii', stop_words="english")
matrix_tfidfvect = tfidf_vectorizer.fit_transform(samsung.text_processed)

# 10 topics. Latent Dirichlet Allocation (LDA)  &  Non-Negative Matrix Factorization (NMF)

In [6]:
# n_samples = 2000
# n_features = 1000
# number_components = 10
number_components = 10
no_top_words = 15
# Function to print the topics
def display_topics(model, feature_names, no_top_words):    
    for topic_idx, topic in enumerate(model.components_):        
        print("Topic %d:" % (topic_idx))       
        print (",".join([feature_names[i]                       
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

## Both models on CV

In [7]:
model_LDA_cv = LatentDirichletAllocation(n_components=number_components)
model_LDA_cv.fit_transform(doc_term_matrix_cv)

array([[0.01428849, 0.34702411, 0.37347068, ..., 0.01428576, 0.01428705,
        0.01428912],
       [0.03333946, 0.37663701, 0.03333775, ..., 0.03333379, 0.03333788,
        0.03334806],
       [0.0200008 , 0.02000516, 0.02000108, ..., 0.02000049, 0.02000118,
        0.81996725],
       ...,
       [0.01250198, 0.44803598, 0.01250048, ..., 0.0125001 , 0.01250042,
        0.45195627],
       [0.01428661, 0.01428759, 0.0142869 , ..., 0.01428638, 0.01428666,
        0.87141564],
       [0.00476243, 0.0047629 , 0.00476225, ..., 0.00476276, 0.00476236,
        0.0047622 ]])

In [8]:
model_NMF_cv = NMF(n_components=number_components)
model_NMF_cv.fit_transform(doc_term_matrix_cv)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00354241, 0.003577  , 0.        , ..., 0.0006637 , 0.        ,
        0.00730012],
       [0.00524717, 0.01833741, 0.00916735, ..., 0.        , 0.        ,
        0.00726601],
       ...,
       [0.        , 0.07418932, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.16331676, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00634581, 0.        , 0.        , ..., 0.        , 0.        ,
        0.30837072]])

In [9]:
print('COUNT VECTORIZER. TWO MODELS')
t0 = time.time()
print('LDA MODEL TOPICS')
display_topics(model_LDA_cv, count_vectorizer.get_feature_names(), no_top_words)
print("Done in %0.3fs." % (time.time() - t0))

print()

print('NMF MODEL TOPICS')
display_topics(model_NMF_cv, count_vectorizer.get_feature_names(), no_top_words)
print("Done in %0.3fs." % (time.time() - t0))

COUNT VECTORIZER. TWO MODELS
LDA MODEL TOPICS
Topic 0:
samsung,note,camera,screen,like,use,battery,apps,good,better,galaxy,iphone,dont,android,fingerprint
Topic 1:
good,stars,condition,perfect,works,excellent,product,great,like,far,happy,came,expected,new,battery
Topic 2:
screen,battery,life,refurbished,ive,poor,note,issues,protector,bad,new,got,used,glass,scratches
Topic 3:
new,like,works,great,brand,looks,refurbished,came,sim,card,got,far,verizon,purchase,old
Topic 4:
unlocked,att,verizon,work,network,locked,tmobile,carrier,sprint,use,compatible,excelente,version,unlock,sim
Topic 5:
screen,samsung,months,working,buy,worked,ok,warranty,stopped,year,broken,time,broke,dropped,month
Topic 6:
sim,card,box,service,seller,charger,samsung,received,came,customer,new,didnt,amazon,device,come
Topic 7:
work,star,charger,gift,didnt,doesnt,got,bought,return,money,stolen,returned,nice,sent,refund
Topic 8:
battery,dont,charge,buy,money,im,bought,time,bad,use,wont,work,got,like,months
Topic 9:
great,

## Both model on Tf-idf vectorizer

In [10]:
model_LDA_tf= LatentDirichletAllocation(n_components=number_components)
model_LDA_tf.fit(matrix_tfidfvect)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [11]:
model_NMF_tf = NMF(n_components=number_components)
model_NMF_tf.fit_transform(matrix_tfidfvect)

array([[0.00000000e+00, 0.00000000e+00, 7.34517515e-02, ...,
        0.00000000e+00, 0.00000000e+00, 2.00565076e-03],
       [0.00000000e+00, 1.34446115e-01, 1.31971459e-04, ...,
        0.00000000e+00, 0.00000000e+00, 4.37058819e-03],
       [2.53583480e-03, 0.00000000e+00, 2.40487923e-03, ...,
        5.80132711e-02, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.67634494e-02, 1.74863715e-04, 1.02792926e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.52455706e-02, 0.00000000e+00, 0.00000000e+00, ...,
        1.59704655e-03, 9.49738483e-04, 7.79192075e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [12]:
print('TF-IDF VECTORIZER. TWO MODELS')
t0 = time.time()
print('LDA MODEL')
display_topics(model_LDA_tf, tfidf_vectorizer.get_feature_names(), no_top_words)
print("Done in %0.3fs." % (time.time() - t0))

print()

print('NMF MODEL')
display_topics(model_NMF_tf, tfidf_vectorizer.get_feature_names(), no_top_words)
print("Done in %0.3fs." % (time.time() - t0))

TF-IDF VECTORIZER. TWO MODELS
LDA MODEL
Topic 0:
screen,great,battery,good,like,samsung,new,camera,life,note,use,love,price,ok,works
Topic 1:
star,work,broken,charger,bad,stars,pen,didnt,broke,unlocked,brother,buy,screen,returned,came
Topic 2:
excelente,bueno,telefono,buen,bien,producto,perfecto,celular,llego,funciona,precio,buena,nuevo,bateria,camara
Topic 3:
best,great,love,easy,good,use,samsung,stars,ive,fantastic,new,galaxy,price,like,size
Topic 4:
new,like,brand,looks,works,great,stars,perfectly,refurbished,scratches,condition,looked,perfect,excelente,came
Topic 5:
loves,gift,great,wife,bought,husband,love,loved,stars,reliable,daughter,mom,good,works,friend
Topic 6:
battery,charge,months,life,buy,working,screen,dont,stopped,work,good,hot,charger,star,bad
Topic 7:
excellent,perfect,stars,thanks,expectations,condition,metro,box,great,yes,durable,pcs,met,headphones,came
Topic 8:
stars,great,good,love,works,product,price,nice,awesome,far,condition,excellent,like,new,happy
Topic 9:
unl

## LDA vizualisation on both count and tfidf vectorizer

In [13]:
pyLDAvis.sklearn.prepare(model_LDA_tf, matrix_tfidfvect, tfidf_vectorizer)

C:\Users\eleouetd01\anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.127559 -0.062743       1        1  27.691826
9      0.120562 -0.149704       2        1  25.005179
8      0.165187  0.188502       3        1  11.558606
6      0.062351 -0.181171       4        1   8.004887
3      0.069889  0.050285       5        1   6.187880
4     -0.003443  0.111644       6        1   4.974703
2     -0.329758  0.006970       7        1   4.536030
5     -0.002347  0.076503       8        1   4.508124
1     -0.111807 -0.117688       9        1   3.953179
7     -0.098193  0.077401      10        1   3.579585, topic_info=      Category         Freq         Term        Total  loglift  logprob
8870   Default  1240.000000        stars  1240.000000  30.0000  30.0000
3981   Default  1475.000000        great  1475.000000  29.0000  29.0000
3894   Default  1174.000000         good  1174.000000  28.0000  28.0000
5921   Default   779.000000          new   779.000000  27.0000  27.0000
8866   Default   324.000000         star   324.000000  26.0000  26.0000
3125   Default   409.000000    excellent   409.000000  25.0000  25.0000
5289   Default   766.000000         love   766.000000  24.0000  24.0000
10428  Default   737.000000        works   737.000000  23.0000  23.0000
6523   Default   397.000000      perfect   397.000000  22.0000  22.0000
5152   Default   669.000000         like   669.000000  21.0000  21.0000
766    Default   603.000000      battery   603.000000  20.0000  20.0000
857    Default   302.000000         best   302.000000  19.0000  19.0000
1058   Default   291.000000        brand   291.000000  18.0000  18.0000
5266   Default   275.000000        looks   275.000000  17.0000  17.0000
5296   Default   160.000000        loves   160.000000  16.0000  16.0000
3121   Default   169.000000    excelente   169.000000  15.0000  15.0000
6984   Default   446.000000        price   446.000000  14.0000  14.0000
1743   Default   352.000000    condition   352.000000  13.0000  13.0000
7066   Default   469.000000      product   469.000000  12.0000  12.0000
1136   Default   117.000000        bueno   117.000000  11.0000  11.0000
3824   Default   124.000000         gift   124.000000  10.0000  10.0000
5956   Default   386.000000         nice   386.000000   9.0000   9.0000
10409  Default   435.000000         work   435.000000   8.0000   8.0000
1431   Default   209.000000       charge   209.000000   7.0000   7.0000
1030   Default   320.000000       bought   320.000000   6.0000   6.0000
664    Default   276.000000      awesome   276.000000   5.0000   5.0000
10323  Default   103.000000         wife   103.000000   4.0000   4.0000
9835   Default   407.000000     unlocked   407.000000   3.0000   3.0000
2327   Default   174.000000    described   174.000000   2.0000   2.0000
9287   Default    89.000000     telefono    89.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
10516  Topic10    26.777815          yes    47.586989   2.7549  -4.8385
5519   Topic10     7.998212        meets    12.736003   2.8647  -6.0468
3125   Topic10   158.211041    excellent   409.745091   2.3783  -3.0621
5588   Topic10    27.130674        metro    58.205278   2.5666  -4.8254
6523   Topic10   127.574833      perfect   397.347923   2.1938  -3.2774
2771   Topic10    24.193821      durable    53.803626   2.5307  -4.9400
9354   Topic10    48.181021       thanks   130.107908   2.3365  -4.2511
4923   Topic10     5.189800          kid     7.929183   2.9061  -6.4794
3009   Topic10     7.786146    equipment    14.100104   2.7361  -6.0737
189    Topic10     6.043296    advertise    10.243492   2.8022  -6.3271
8870   Topic10   114.297213        stars  1240.031731   0.9458  -3.3873
5087   Topic10     7.332102        legit    14.334366   2.6595  -6.1338
3465   Topic10    12.880459          fit    37.548947   2.2600  -5.5703
4178   Topic10    17.522328   headphones    64.643616   2.0245  -5.26

In [17]:
pyLDAvis.sklearn.prepare(model_LDA_cv, doc_term_matrix_cv, count_vectorizer)
%time

Wall time: 0 ns


C:\Users\eleouetd01\anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
